In [ ]:
!pip install --upgrade -q git+https://github.com/google/flax.git

     |████████████████████████████████| 145 kB 3.2 MB/s 
     |████████████████████████████████| 217 kB 27.2 MB/s 
     |████████████████████████████████| 596 kB 59.7 MB/s 
     |████████████████████████████████| 51 kB 7.4 MB/s 
     |████████████████████████████████| 76 kB 5.4 MB/s 


In [ ]:
!pip install -U jax[cuda11_cudnn83] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


In [ ]:
import matplotlib.pyplot as plt
from jax import lax, random, numpy as jnp
from jax import grad, jit, vmap

import flax
from flax.core import freeze, unfreeze
from flax import linen as nn
from flax.training import train_state

import os
import numpy as np
from google.colab import drive
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import torchvision
rng = np.random.default_rng(12345)
# drive.mount('/content/drive/')

In [ ]:
!XLA_PYTHON_CLIENT_PREALLOCATE=false

In [ ]:
from jax import jit

In [ ]:
background_data = torchvision.datasets.Omniglot(
    root="./data",background=True, download=True, transform=torchvision.transforms.ToTensor()
)

total_images = 19280

x = []
imgs = []
for i in range(0,19280):
  img, label = background_data[i]
  del label
  imgs.append(jnp.array(img[0]))
  del img
  if (i+1)%20 == 0:
    print(i)
    x.append(jnp.stack(imgs))
    imgs = []
x = jnp.stack(x)
x.shape
del background_data

  0%|          | 0/9464212 [00:00<?, ?it/s]

Extracting ./data/omniglot-py/images_background.zip to ./data/omniglot-py
19
39
59
79
99
119
139
159
179
199
219
239
259
279
299
319
339
359
379
399
419
439
459
479
499
519
539
559
579
599
619
639
659
679
699
719
739
759
779
799
819
839
859
879
899
919
939
959
979
999
1019
1039
1059
1079
1099
1119
1139
1159
1179
1199
1219
1239
1259
1279
1299
1319
1339
1359
1379
1399
1419
1439
1459
1479
1499
1519
1539
1559
1579
1599
1619
1639
1659
1679
1699
1719
1739
1759
1779
1799
1819
1839
1859
1879
1899
1919
1939
1959
1979
1999
2019
2039
2059
2079
2099
2119
2139
2159
2179
2199
2219
2239
2259
2279
2299
2319
2339
2359
2379
2399
2419
2439
2459
2479
2499
2519
2539
2559
2579
2599
2619
2639
2659
2679
2699
2719
2739
2759
2779
2799
2819
2839
2859
2879
2899
2919
2939
2959
2979
2999
3019
3039
3059
3079
3099
3119
3139
3159
3179
3199
3219
3239
3259
3279
3299
3319
3339
3359
3379
3399
3419
3439
3459
3479
3499
3519
3539
3559
3579
3599
3619
3639
3659
3679
3699
3719
3739
3759
3779
3799
3819
3839
3859
3879
3899
3919
3

In [ ]:
def batch(batch_size, x):
  classes, examples, w, h = x.shape
  categories = rng.choice(classes,size=(batch_size,),replace=False)
  pairs=[np.zeros((batch_size, h, w,1)) for i in range(2)]
  targets=np.zeros((batch_size,))

  targets[batch_size//2:] = 1
  for i in range(batch_size):
    category = categories[i]
    idx_1 = rng.integers(0, examples)
    pairs[0][i,:,:,:] = x[category, idx_1].reshape(w, h, 1)
    idx_2 = rng.integers(0, examples)

    if i >= batch_size // 2:
      category_2 = category
    else:
      category_2 = (category +rng.integers(1,classes)) % classes

    pairs[1][i,:,:,:] = x[category_2,idx_2].reshape(w, h,1)

  return pairs, targets

In [ ]:
"""def loadData(path):
  x = []
  y = []
  lang_info = {}
  current_y = 0
  count = 0
  for language in os.listdir(path):
    language_path = os.path.join(path,language)
    lang_info[language] = [current_y,None]

    for letter in os.listdir(language_path):
      letter_path = os.path.join(language_path,letter)
      img_category = []

      for filename in os.listdir(letter_path):
        image_path = os.path.join(letter_path, filename)
        img, label = background_data[count]
        img_category.append(jnp.array(img[0]))
        del img
        y.append(current_y)
        count += 1
      x.append(jnp.stack(img_category))
      temp = jnp.stack(x)
      current_y += 1
      lang_info[language][1] = current_y - 1
    if count == 5360:
       y = jnp.vstack(y)
       x = jnp.stack(x)
       return x,y,lang_info
    print(language)
    print(temp.shape)
  y = jnp.vstack(y)
  x = jnp.stack(x)
  return x,y,lang_info
x,y,lang_info = loadData('/content/drive/MyDrive/omniglot/images_background')
del background_data
del y"""

"def loadData(path):\n  x = []\n  y = []\n  lang_info = {}\n  current_y = 0\n  count = 0\n  for language in os.listdir(path):\n    language_path = os.path.join(path,language)\n    lang_info[language] = [current_y,None]\n\n    for letter in os.listdir(language_path):\n      letter_path = os.path.join(language_path,letter)\n      img_category = []\n\n      for filename in os.listdir(letter_path):\n        image_path = os.path.join(letter_path, filename)\n        img, label = background_data[count]\n        img_category.append(jnp.array(img[0]))\n        del img\n        y.append(current_y)\n        count += 1\n      x.append(jnp.stack(img_category))\n      temp = jnp.stack(x)\n      current_y += 1\n      lang_info[language][1] = current_y - 1\n    if count == 5360:\n       y = jnp.vstack(y)\n       x = jnp.stack(x)\n       return x,y,lang_info\n    print(language)\n    print(temp.shape)\n  y = jnp.vstack(y)\n  x = jnp.stack(x)\n  return x,y,lang_info\nx,y,lang_info = loadData('/content/d

In [ ]:
class SNN(nn.Module):

    def setup(self):
        self.conv1 = nn.Conv(features=64, kernel_size=(10,10), padding="SAME", name="CONV1")
        self.conv2 = nn.Conv(features=128, kernel_size=(7,7), padding="SAME", name="CONV2")
        self.conv3 = nn.Conv(features=128, kernel_size=(4,4), padding="SAME", name="CONV3")
        self.conv4 = nn.Conv(features=256, kernel_size=(4,4), padding="SAME", name="CONV4")
        self.linear1 = nn.Dense(4096, name="DENSE")
        self.linear2 = nn.Dense(1, name="OUTPUT")

    @nn.compact
    def __call__(self, input1, input2):

        x1 = nn.relu(self.conv1(input1))
        x1 = nn.max_pool(x1, window_shape=(2, 2), strides=(2,2))
        x1 = nn.relu(self.conv2(x1))
        x1 = nn.max_pool(x1, window_shape=(2, 2), strides=(2,2))
        x1 = nn.relu(self.conv3(x1))
        x1 = nn.max_pool(x1, window_shape=(2, 2), strides=(2,2))
        x1 = nn.relu(self.conv4(x1))
        x1 = x1.reshape((x1.shape[0], -1))
        #x1 = jnp.ravel(x1)
        x1 = nn.sigmoid(self.linear1(x1))

        x2 = nn.relu(self.conv1(input2))
        x2 = nn.max_pool(x2, window_shape=(2, 2), strides=(2,2))
        x2 = nn.relu(self.conv2(x2))
        x2 = nn.max_pool(x2, window_shape=(2, 2), strides=(2,2))
        x2 = nn.relu(self.conv3(x2))
        x2 = nn.max_pool(x2, window_shape=(2, 2), strides=(2,2))
        x2 = nn.relu(self.conv4(x2))
        x2 = x2.reshape((x2.shape[0], -1))
        #x2 = jnp.ravel(x2)
        x2 = nn.sigmoid(self.linear1(x2))

        dX = jnp.abs(x2-x1)
        dX = nn.sigmoid(self.linear2(dX))
        return dX

In [ ]:
seed = random.PRNGKey(0)
subseed,seed = random.split(seed)
model = SNN()
x1, y1 = batch(8,x)
xa = jnp.array(x1[0])
xb = jnp.array(x1[1])
params = model.init(seed, xa, xb)
print(SNN().tabulate(seed, xa, xb))

                           SNN Summary                            
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path   ┃ outputs                ┃ params                       ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Inputs │ - float32[8,105,105,1] │                              │
│        │ - float32[8,105,105,1] │                              │
├────────┼────────────────────────┼──────────────────────────────┤
│ CONV1  │ float32[8,105,105,64]  │ bias: float32[64]            │
│        │                        │ kernel: float32[10,10,1,64]  │
│        │                        │                              │
│        │                        │ 6,464 (25.9 KB)              │
├────────┼────────────────────────┼──────────────────────────────┤
│ CONV2  │ float32[8,52,52,128]   │ bias: float32[128]           │
│        │                        │ kernel: float32[7,7,64,128]  │
│        │                        │                              │
│        │                        │ 401,536 (1.6 MB)             │
├────────┼────────────────────────┼──────────────────────────────┤
│ CONV3  │ float32[8,26,26,128]   │ bias: float32[128]           │
│        │                        │ kernel: float32[4,4,128,128] │
│        │                        │                              │
│        │                        │ 262,272 (1.0 MB)             │
├────────┼────────────────────────┼──────────────────────────────┤
│ CONV4  │ float32[8,13,13,256]   │ bias: float32[256]           │
│        │                        │ kernel: float32[4,4,128,256] │
│        │                        │                              │
│        │                        │ 524,544 (2.1 MB)             │
├────────┼────────────────────────┼──────────────────────────────┤
│ DENSE  │ float32[8,4096]        │ bias: float32[4096]          │
│        │                        │ kernel: float32[43264,4096]  │
│        │                        │                              │
│        │                        │ 177,213,440 (708.9 MB)       │
├────────┼────────────────────────┼──────────────────────────────┤
│ OUTPUT │ float32[8,1]           │ bias: float32[1]             │
│        │                        │ kernel: float32[4096,1]      │
│        │                        │                              │
│        │                        │ 4,097 (16.4 KB)              │
├────────┼────────────────────────┼──────────────────────────────┤
│ SNN    │ float32[8,1]           │                              │
├────────┼────────────────────────┼──────────────────────────────┤
│        │                  Total │ 178,412,353 (713.6 MB)       │
└────────┴────────────────────────┴──────────────────────────────┘
                                                                  
             Total Parameters: 178,412,353 (713.6 MB)             

In [ ]:
def CrossEntropyLoss(weights, input1, input2, actual):
  preds = model.apply(weights, input1, input2)
  one_hot_actual = nn.one_hot(actual, num_classes = 2)
  log_preds = jnp.log(preds)
  return - jnp.sum(one_hot_actual*log_preds)

In [ ]:
pred = model.apply(params, xa, xb)
print(pred)
print(y1)

[[0.5054711 ]
 [0.5043207 ]
 [0.5039922 ]
 [0.5017871 ]
 [0.5006657 ]
 [0.4991673 ]
 [0.5005816 ]
 [0.49984184]]
[0. 0. 0. 0. 1. 1. 1. 1.]


In [ ]:
from jax import value_and_grad
import optax
from flax import optim
optimizer = optax.sgd(learning_rate=0.005)
optimizer_state = optimizer.init(params)

@jit
def train_model(x1,x2, y, weights, optimizer_state):
  losses = []

  loss, gradients = value_and_grad(CrossEntropyLoss)(weights, x1, x2, y)
  updates, optimizer_states = optimizer.update(gradients, optimizer_state)
  weights = optax.apply_updates(weights, updates)
  losses.append(loss)

  return weights,loss

In [ ]:
for i in range(20000):
  params,loss = train_model(xa,xb,y1,params,optimizer_state)
  train_model._clear_cache()
  print(f"Loss (Iteration {i}): {loss}")

Streaming output truncated to the last 5000 lines.
Loss (Iteration 15000): 2.348429916310124e-05
Loss (Iteration 15001): 2.348429916310124e-05
Loss (Iteration 15002): 2.348429916310124e-05
Loss (Iteration 15003): 2.348429916310124e-05
Loss (Iteration 15004): 2.348429916310124e-05
Loss (Iteration 15005): 2.348429916310124e-05
Loss (Iteration 15006): 2.336508987355046e-05
Loss (Iteration 15007): 2.336508987355046e-05
Loss (Iteration 15008): 2.336508987355046e-05
Loss (Iteration 15009): 2.336508987355046e-05
Loss (Iteration 15010): 2.336508987355046e-05
Loss (Iteration 15011): 2.336508987355046e-05
Loss (Iteration 15012): 2.336508987355046e-05
Loss (Iteration 15013): 2.336508987355046e-05
Loss (Iteration 15014): 2.336508987355046e-05
Loss (Iteration 15015): 2.336508987355046e-05
Loss (Iteration 15016): 2.336508987355046e-05
Loss (Iteration 15017): 2.336508987355046e-05
Loss (Iteration 15018): 2.336508987355046e-05
Loss (Iteration 15019): 2.336508987355046e-05
Loss (Iteration 15020): 2.336

In [ ]:
pred = model.apply(params, xa, xb)
print(pred)
print(y1)

[[0.999987  ]
 [0.9999937 ]
 [0.9999795 ]
 [0.9999877 ]
 [0.9999925 ]
 [0.9999832 ]
 [0.99994636]
 [0.99996376]]
[0. 0. 0. 0. 1. 1. 1. 1.]


In [ ]:
def testing_batch(N,x):
  classes,examples,width,height = x.shape
  letter = np.random.randint(0,964)
  letter_example = np.random.randint(0,20)
  support_set = rng.choice(964,size=(N-1,),replace=False)
  while letter in support_set:
    support_set = rng.choice(964,size=(N-1,),replace=False)
  support_set = random.shuffle(random.PRNGKey(np.random.randint(100)),jnp.append(support_set,letter))
  test_image = x[letter, letter_example:letter_example+1,:,:]
  return test_image, support_set, letter

In [ ]:
image, support, letter = testing_batch(9,x)
print(support)
print(letter)
temp1 = []
temp2 = []
for index in support:
  letter_example = np.random.randint(0,20)
  support_image = x[index, letter_example:letter_example+1,:,:]
  temp1.append(image[0].reshape(105,105,1))
  temp2.append(support_image[0].reshape(105,105,1))
temp1 = jnp.array(temp1)
temp2 = jnp.array(temp2)
result = model.apply(params,temp1,temp2)
print(result)
print(jnp.argmax(result))

#just have to train it now lol

/usr/local/lib/python3.7/dist-packages/jax/_src/random.py:372: FutureWarning: jax.random.shuffle is deprecated and will be removed in a future release. Use jax.random.permutation with independent=True.
  warnings.warn(msg, FutureWarning)


[467 265 654 408 446 501 368 119  29]
368
[[0.9972153 ]
 [0.9999939 ]
 [0.99727833]
 [0.9998566 ]
 [0.99937046]
 [0.99995947]
 [0.99998593]
 [0.99992025]
 [0.99991214]]
1


In [ ]:
def at_index(arr, element):
  count = 0
  for el in arr:
    if el == element:
      return count
    count+=1

In [ ]:
count = 0
c1 = 0
tests = 100
for i in range(tests):
  image, support, letter = testing_batch(4,x)
  temp1 = []
  temp2 = []
  for index in support:
    letter_example = np.random.randint(0,20)
    support_image = x[index, letter_example:letter_example+1,:,:]
    temp1.append(image[0].reshape(105,105,1))
    temp2.append(support_image[0].reshape(105,105,1))
  temp1 = jnp.array(temp1)
  temp2 = jnp.array(temp2)
  result = model.apply(params,temp1,temp2)
  if jnp.argmax(result) == at_index(support,letter):
    count+=1
  c1+=1
  if c1%5 ==0:
    print(c1)
print(f"Accuracy: {(count/tests)*100}%")

/usr/local/lib/python3.7/dist-packages/jax/_src/random.py:372: FutureWarning: jax.random.shuffle is deprecated and will be removed in a future release. Use jax.random.permutation with independent=True.
  warnings.warn(msg, FutureWarning)


5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
Accuracy: 8.0%
